TO DO:
- Implement differences between efficiency/power curve PSDs
- Update populated curve PSD to reflect Diameter units (Cell B3), Power (B4), Pwr/Eta cell (B5) 

##### Imports, File Setup

In [147]:
import os
import pandas as pd
import shutil
from openpyxl import load_workbook
import re

In [148]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\15 SP Integration\Curve PSD"
myFile = "PumpCurves.csv"
filePath = os.path.join(myDir, myFile)

# This creates a dataframe of the curve export csv, and fills in the RPM(curve nominal) column
data = pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False)
data = data.replace(',','.', regex=True)
data['RPM(Curve nominal)'] = data['RPM(Curve nominal)'].ffill()

In [149]:
# This points to the curve PSD template to be used
templateDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\30-39 Resources\32 GXS"
template = "SKB Blank Curve PSD - Efficiency_Metric.xlsx"
template = os.path.join(templateDir, template)

# Create a local working copy to leave template unmodified
workingCopy = os.path.join(myDir, "Populated Curve PSD -imp diameters.xlsx")
shutil.copyfile(template, workingCopy)
wb = load_workbook(workingCopy)      

##### Functions

In [150]:
# def get_min_max_values(myList):

#     min_speed = myList[-1]
#     max_speed = myList[0]
#     max_nominal = max_speed

#     return min_speed, max_nominal, max_speed

In [151]:
# def findSpeedCells(my_df):
#     """ function to find which cell to start populating curve data based on RPM"""
#     # from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
#     from openpyxl.utils.cell import get_column_letter
#     speedCells = []
    
#     max_cols = 21 * len(my_df.RPM.unique())
#     row = 7    
#     first_col = 4
#     diameter_cols = list(range(first_col,max_cols,21))
    
#     for item in diameter_cols:
#         col = get_column_letter(item)
#         cell_coordinate = "{}{}".format(col,row)
#         speedCells.append(cell_coordinate)      

#     return(speedCells)

##### Main

In [152]:
def curveDataByPartNumber(data) -> dict:
    """Creates dictionary of dataframes for each PN """
    import math
    
    # This will become a dictionary with keys=pns and values=curve_data_dataframes
    dict_curveDataByPn = {}
    
    # Iterate through each row in pump curve data export 
    for index, value in data.iterrows():    
        
        # When a PN is encountered:
        # if not math.isnan(value['ProductNumber']):
        #     currentProductNumber = int(value['ProductNumber'])
        if type(value['ProductNumber']) == str:
            currentProductNumber = value['ProductNumber']
            
            # Resets list for each PN
            listOfFlows = []
            listOfHeads = []
            listOfPow = []
            listOfSpeeds = []
            listOfNPSH = []
            
        # Recording Q/H values to lists
        if value['RPM(Curve nominal)'] > 0:
            listOfFlows.append(value['Q'])
            listOfHeads.append(value['H'])
            listOfNPSH.append(value['NPSH'])
            listOfPow.append(value['P1'])
            listOfSpeeds.append(value['RPM(Curve nominal)'])
            
        # At end of Q/H values, store in dataframe before moving to next PN, or when end of data is reached
        if pd.isna(value['Q']) and pd.isna(value['H']) or (index == len(data)-1):    
            zipped = list(zip(listOfFlows, listOfHeads, listOfPow, listOfNPSH, listOfSpeeds))
            df = pd.DataFrame(zipped, columns=['Q','H','P1', 'NPSH','RPM'])

            # Drop rows that have NaNs, then add df to dictionary
            df = df.dropna()
            dict_curveDataByPn.update({currentProductNumber:df})

            continue

    return dict_curveDataByPn
    
# Creates dictionary with part numbers as keys, curves as dataframes for each key
curveDataDict = curveDataByPartNumber(data)    

In [153]:
def curveDataMatches(curveDataDict):  
    """ Create list of PNs that share curve data """

    curveDataCopy = curveDataDict.copy()
    listOfLists = []
    
    for eachPN, eachDF in curveDataDict.items():  
        listOfPNs = []
        for key, value in curveDataCopy.items():
            if value.equals(eachDF):
                listOfPNs.append(key)
        
        if listOfPNs not in listOfLists:
            listOfLists.append(listOfPNs)
    
    # print(f'list of lists: {listOfLists}')
    # print(f'list of pns: {listOfPNs}')
    
    return listOfLists
    
# Creates list of which part numbers share curve data
uniqueCurvePartnumbers = curveDataMatches(curveDataDict)

In [154]:
def add_a_curve(list_unique_curves):  # WORKING. DO NOT MODIFY YET
    """ CREATE new tabs for each unique curve family """
    
    global wb
    tab_to_copy = wb['NEW'] 
    
    partnumbers = []
    min_speeds = []
    nom_speeds = []
    max_speeds = []
    
    # Add a tab for each unique curve
    for item in list_unique_curves: 
        
        list_of_speeds = []

        # Add 1 tab
        tabName = str(item[0])
        # print(tabName)
        wb.copy_worksheet(tab_to_copy).title = tabName

        # Fill in all speeds/trims in column A
        for key, value in curveDataDict[tabName].iterrows():
            list_of_speeds.append(value['RPM'])
        
        speed_set = sorted(set(list_of_speeds), reverse=True)
        # print(tabName, speed_set)  
        
        for index, eachSpeed in enumerate(speed_set):
            cell_name = "{}{}".format('A', 10+index)
            curveSheet = wb[tabName]
            curveSheet[cell_name].value = int(eachSpeed)

            min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)
            
            partnumbers.append(tabName)
            min_speeds.append(min_rpm)
            nom_speeds.append(nom_rpm)
            max_speeds.append(max_rpm)
            
            zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
            df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
        
    return df
    
# Adds curve tabs for each unique curve
# minmax_speeds_df = add_a_curve(uniqueCurvePartnumbers)

In [155]:
# Add 1 curve tab to workbook for every unique curve found (avoids duplicating data in SKB)
for item in uniqueCurvePartnumbers: 
    tabName = str(item[0])
    wb.copy_worksheet(wb['NEW']).title = tabName # Creates and renames blank PSD Tab as template for each new curve tab

In [ ]:
def extract_variable_speeds(df) -> list:
    ''' Takes curve data in dataframe format, and returns list of all unique speeds for variable speed pumps'''
    list_of_rpms = []

    for key, value in df.iterrows():
        list_of_rpms.append(value['RPM'])
    
    list_of_rpms = sorted(set(list_of_rpms), reverse=True)

    return list_of_rpms

# Iterate through list of unique curve names, and populate the corresponding workbook tab
for uniqueCurve in uniqueCurvePartnumbers:
    curveSheet = wb[uniqueCurve[0]]
    curve_data_df = curveDataDict[uniqueCurve[0]]
    
    # Fill Speed or trim in column A
    speed_set = extract_variable_speeds(curve_data_df)

    for index, eachSpeed in enumerate(speed_set):
        cell_name = "{}{}".format('A', 10+index)
        curveSheet[cell_name].value = int(eachSpeed)
        print(f'For PN: {uniqueCurve[0]} -> cell: {cell_name} should have value: {int(eachSpeed)}. It currently has: {curveSheet[cell_name].value}')

    # Fill Curve, power/eff, NPSH data
    

In [157]:
# def speeds_for_curve_header_data():
#         # partnumbers = []
#         # min_speeds = []
#         # nom_speeds = []
#         # max_speeds = []
        
#         min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)
        
#         partnumbers.append(tabName)
#         min_speeds.append(min_rpm)
#         nom_speeds.append(nom_rpm)
#         max_speeds.append(max_rpm)
        
#         zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
#         df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
#     return

In [158]:
def fillCurveHeaderDataTab(df_of_speeds):
    global wb
    newPumpFamilyName = "SP"
    sheet = wb["Curve Header Data"]

    cellName = 'A1' 
    first_row_offset = 10

    for index, speedData in df_of_speeds.iterrows():
        sheet[cellName].offset((first_row_offset + index), 0).value = speedData['Partnumber']
        sheet[cellName].offset((first_row_offset + index), 2).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 6).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 7).value = speedData['Min RPM']
        sheet[cellName].offset((first_row_offset + index), 8).value = speedData['Max RPM']

    sheet['B7'] = newPumpFamilyName
    wb.save(workingCopy)

    return

# fillCurveHeaderDataTab(minmax_speeds_df)

In [159]:
def addCurveFamiliesTab():
    """ Create list of PNs that share curve data, and inserts tab to illustrate """
    
    global wb    
    curveFamilySheet = wb.create_sheet("Shared Curves", 2)
    
    for index, family in enumerate(uniqueCurvePartnumbers):
        row = index + 1  # Excel doesn't like 0-indexes

        # Fill 1st column in spreadsheet
        curveFamilySheet.cell(row=row, column=1).value = "Curve " + str(row)

        # Fill 2nd (or more) columns with partnumbers that share curves
        if len(family) == 1:
            curveFamilySheet.cell(row=row, column=2).value = family[0]
            # print('length of family is 1')
        else:
            curveFamilySheet.cell(row=row, column=2).value = family[0]
            curveFamilySheet.cell(row=row, column=3).value = family[1]
            # print(f'family[0]: {family[0]}, family[1]: {family[1]}')

    return
    
# addCurveFamiliesTab()

In [160]:
def curveFiller(pn_vs_curves_dict):
    """ fills curve tables in each tab """
    global wb
    cellName = 'D7' 
    first_row_offset = 3

    # Iterate through each tab in workbook
    for sheet in wb.worksheets:
        try:
            sheetname = int(sheet.title)
        except ValueError:
            sheetname = sheet.title

        print(f'Current Sheet: {sheetname}')

        # Only process tabs that contain curve data
        for model_name, curve_dataframe in pn_vs_curves_dict.items():             
            if model_name == sheetname:
                print(f'Using curve dataframe for: {model_name}')
                print(pn_vs_curves_dict[model_name])
                # Iterate through dataframe row by row and fill out each row in PSD    
                for key, value in curve_dataframe.iterrows():
                    sheet[cellName].offset(first_row_offset + key, 0).value = value['Q']
                    sheet[cellName].offset(first_row_offset + key, 1).value = value['H']
                    sheet[cellName].offset(first_row_offset + key, 7).value = value['Q']
                    # sheet[cellName].offset(first_row_offset + key, 8).value = value['Eta1']
                    sheet[cellName].offset(first_row_offset + key, 8).value = value['P1']
                    sheet[cellName].offset(first_row_offset + key,14).value = value['Q']
                    sheet[cellName].offset(first_row_offset + key,15).value = value['NPSH']     

        # print(list(sheet.values))

    return   
    
# Fills each new curve tab
# curveFiller(curveDataDict)

In [161]:
# Save changes to excel sheet
wb.save(workingCopy)